In [166]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
import re



from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score


import joblib

# Обзор данных

исходные данные - таблица с разным количеством столбцов. Выделим только строки, соответствующие заданному формату данных

In [167]:
# Список для хранения корректных строк 83337

t = []

with open('data/train.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)

    for row in reader:
        # Проверяем, что первая колонка существует и начинается с "20"
        if row and row[0].startswith("20"):  # Проверка на пустую строку
            t.append(row)

# Создаем DataFrame из списка корректных строк
df = pd.DataFrame(t, columns=header)

# Вывод информации о DataFrame

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83411 entries, 0 to 83410
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             83411 non-null  object
 1   reel_name        83411 non-null  object
 2   yt_reel_id       83411 non-null  object
 3   cartoon          83411 non-null  object
 4   url              83411 non-null  object
 5   text             83411 non-null  object
 6   seconds          83338 non-null  object
 7   is_shorts        83338 non-null  object
 8   broadcast        83338 non-null  object
 9   yt_channel_id    83338 non-null  object
 10  yt_channel_name  83338 non-null  object
 11  yt_ch_url        83338 non-null  object
 12  yt_channel_type  83338 non-null  object
 13  flag_closed      83338 non-null  object
 14  international    83338 non-null  object
 15  language         83338 non-null  object
dtypes: object(16)
memory usage: 10.2+ MB


Целевая переменная - наименования проекта - у нас 44 проекта и none

In [168]:
df['cartoon'].unique(), df['cartoon'].describe(), 

(array(['none', 'Смешарики', 'Чуддики', 'Маша и медведь',
        'Кошечки собачки', 'Зебра в клеточку', 'Сумка', 'Фиксики',
        'Бэтмен', 'Енотки', 'Черепашки Ниндзя', 'Свинка Пеппа',
        'ЖилаБыла Царевна', 'My little pony', 'Чик-Чирикино', 'Царевны',
        'Котик Мормотик', 'Буба', 'Малышарики', 'Синий трактор',
        'Говорящий Том', 'Чучело-Мяучело', 'Оранжевая корова',
        'Трансформеры', 'Пороро', 'Акуленок', 'Простоквашино', 'Цветняшки',
        'Кукутики', 'Приключения Пети и Волка', 'Губка Боб',
        'Щенячий патруль', 'Enchantimals (Эншантималс)', 'Барбоскины',
        'Симпсоны', 'Чебурашка', 'Крутиксы', 'Финник',
        'Cry babies magic tears', 'Вспыш', 'Лунтик', 'Мини-мишки',
        'Бременские музыканты', 'Паровозики Чаттингтон',
        'Ну_погоди каникулы'], dtype=object),
 count     83411
 unique       45
 top        none
 freq      78231
 Name: cartoon, dtype: object)

In [169]:

df

,date,reel_name,yt_reel_id,cartoon,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
0,2023-12-30,100 подарков и только один можно оставить себе,PyGZL_fCbSc,none,https://www.youtube.com/watch?v=PyGZL_fCbSc,[100 подарков и только один можно оставить себе],1854.0,0.0,none,UCcartHVtvAUzfajflyeT_Gg,Miss Katy,https://www.youtube.com/channel/UCcartHVtvAUzf...,Блогеры,0.0,0.0,
1,2018-06-29,ЗАБАГАЛИ Лизу в Реальной Жизни АРТУР и НАСТЯ F...,Lcp0pFZF4eQ,none,https://www.youtube.com/watch?v=Lcp0pFZF4eQ,[ЗАБАГАЛИ Лизу в Реальной Жизни АРТУР и НАСТЯ ...,602.0,0.0,none,UCWI7puXzGTPuxsoW2HD3XxQ,Boys and Toys,https://youtube.com/channel/UCWI7puXzGTPuxsoW2...,Блогеры,0.0,0.0,
2,2023-11-26,Грозная семейка Лучшие СЕСТРИНСКИЕ моменты с...,z8d6LbiBRnk,none,https://www.youtube.com/watch?v=z8d6LbiBRnk,[Грозная семейка Лучшие СЕСТРИНСКИЕ моменты ...,3604.0,0.0,none,UCw3vK8lNe5SZzL--rMgq-CQ,Nickelodeon_Cyrillic,https://youtube.com/channel/UCw3vK8lNe5SZzL--r...,Мультфильмы,0.0,0.0,
3,2016-05-30,Как сделать ожерелье из ткани своими руками,0iu0p_Y8DjM,none,https://www.youtube.com/watch?v=0iu0p_Y8DjM,[Как сделать ожерелье из ткани своими руками] ...,264.0,0.0,none,UCq1JpGFxcZTbbOAz010U-og,Трум Трум,https://youtube.com/channel/UCq1JpGFxcZTbbOAz0...,Блогеры,0.0,0.0,
4,2023-07-06,Sleepover Rules Om Nom Stories Cartoon for k...,rYwOVp_Edjw,none,https://www.youtube.com/watch?v=rYwOVp_Edjw,[Sleepover Rules Om Nom Stories Cartoon for ...,42899.0,0.0,none,UCniI-BQk7qAtXNmmz40LSdg,Kedoo Toons TV Funny Animations,https://youtube.com/channel/UCniI-BQk7qAtXNmmz...,Мультфильмы,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83406,2022-09-05,"Уже завтра на IVI a href""https://www.ivi.ru/wa...",eGDeXhMuq4U,none,https://www.youtube.com/watch?v=eGDeXhMuq4U,"[Уже завтра на IVI a href""https://www.ivi.ru/w...",28.0,0.0,none,UCmCK7DVS30UnjZJq2McDuXA,Аминка Витаминка,https://youtube.com/channel/UCmCK7DVS30UnjZJq2...,Блогеры,0.0,0.0,
83407,2024-02-20,Проще простого,htqPiLWfxeI,none,https://www.youtube.com/watch?v=htqPiLWfxeI,[Проще простого] Дракоша Мисоля старательно ри...,15.0,0.0,none,UCID3-FBQUma_vl1a3ktRKWg,Дракоша Тоша,https://youtube.com/channel/UCID3-FBQUma_vl1a3...,Мультфильмы,0.0,0.0,
83408,2011-07-01,Добрыня Никитич и Змей Горыныч Фильм о мультф...,m3p8w04zjTI,none,https://www.youtube.com/watch?v=m3p8w04zjTI,[Добрыня Никитич и Змей Горыныч Фильм о мульт...,1256.0,0.0,none,UCVXEfYCFdyQ9nkoNJY-uYWA,Три Богатыря,https://www.youtube.com/channel/UCVXEfYCFdyQ9n...,Мультфильмы,0.0,0.0,
83409,2020-04-08,Сеня и Папа убегают от Динозавров. Веселые ист...,14OOn0X2ZU0,none,https://www.youtube.com/watch?v=14OOn0X2ZU0,[Сеня и Папа убегают от Динозавров. Веселые ис...,674.0,0.0,none,UCTn9Vyy-3fzLIr0bqhIbNig,Super Senya RU,https://www.youtube.com/channel/UCTn9Vyy-3fzLI...,Блогеры,0.0,0.0,


In [170]:
df['cartoon'].value_counts().sort_values(ascending=True)[:25]

cartoon
Вспыш                          3
Enchantimals (Эншантималс)     4
Паровозики Чаттингтон          6
Бременские музыканты           6
Котик Мормотик                 6
Цветняшки                      6
Барбоскины                     7
Ну_погоди каникулы             7
Чебурашка                      8
Мини-мишки                     8
Финник                         8
Губка Боб                     10
Лунтик                        10
Симпсоны                      10
Акуленок                      11
Кукутики                      17
Крутиксы                      22
Cry babies magic tears        22
Щенячий патруль               32
Царевны                       32
Простоквашино                 35
Трансформеры                  40
Смешарики                     41
Приключения Пети и Волка      41
Пороро                        51
Name: count, dtype: int64

In [171]:
minor_classes = df['cartoon'].value_counts().sort_values(ascending=True)[:25].index
minor_classes

Index(['Вспыш', 'Enchantimals (Эншантималс)', 'Паровозики Чаттингтон',
       'Бременские музыканты', 'Котик Мормотик', 'Цветняшки', 'Барбоскины',
       'Ну_погоди каникулы', 'Чебурашка', 'Мини-мишки', 'Финник', 'Губка Боб',
       'Лунтик', 'Симпсоны', 'Акуленок', 'Кукутики', 'Крутиксы',
       'Cry babies magic tears', 'Щенячий патруль', 'Царевны', 'Простоквашино',
       'Трансформеры', 'Смешарики', 'Приключения Пети и Волка', 'Пороро'],
      dtype='object', name='cartoon')

In [172]:
#спарсим некоторые данные по минимально представленным классам и загрузим их
df2 = pd.read_csv('parse_data.csv')
df2

,reel_name,cartoon
0,Вспыш и чудо-машинки | Вспыш трансформируется ...,Вспыш
1,"Вспыш и чудо-машинки | Вспыш спасает щенка, чу...",Вспыш
2,Вспыш и чудо-машинки | Время гонок! Часть 2 | ...,Вспыш
3,Вспыш и чудо-машинки | Вспыш использует науку ...,Вспыш
4,Вспыш и чудо-машинки | Вспыш гоняет по пути со...,Вспыш
...,...,...
803,ALTERNATIVE MY LITTLE PONY UNIVERSES,My little pony
804,"Kider eggs My Little Pony, SpongeBob, miraculo...",My little pony
805,ЛЕДИ БАГ И СУПЕР-КОТ | 🐞 СБОРНИК День Героев 🐞...,My little pony
806,МОЙ МАЛЕНЬКИЙ ПОНИ | 8 СЕЗОН ПОЛНОСТЬЮ | ЧАСТЬ...,My little pony


Названия проектов закодируем 

In [173]:
target = df['cartoon']
label_encoder = LabelEncoder()
encoded_target_array = label_encoder.fit_transform(target)

# Сохраняем соответствие между оригинальными и закодированными значениями
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Создаем DataFrame из закодированных значений
target_encoded = pd.DataFrame(encoded_target_array, columns=['encoded_cartoon'])


print("\nСоответствие:")
print(mapping)


# decoded_df = pd.DataFrame(target_decoded, columns=['decoded_cartoon'])




Соответствие:
{'Cry babies magic tears': 0, 'Enchantimals (Эншантималс)': 1, 'My little pony': 2, 'none': 3, 'Акуленок': 4, 'Барбоскины': 5, 'Бременские музыканты': 6, 'Буба': 7, 'Бэтмен': 8, 'Вспыш': 9, 'Говорящий Том': 10, 'Губка Боб': 11, 'Енотки': 12, 'ЖилаБыла Царевна': 13, 'Зебра в клеточку': 14, 'Котик Мормотик': 15, 'Кошечки собачки': 16, 'Крутиксы': 17, 'Кукутики': 18, 'Лунтик': 19, 'Малышарики': 20, 'Маша и медведь': 21, 'Мини-мишки': 22, 'Ну_погоди каникулы': 23, 'Оранжевая корова': 24, 'Паровозики Чаттингтон': 25, 'Пороро': 26, 'Приключения Пети и Волка': 27, 'Простоквашино': 28, 'Свинка Пеппа': 29, 'Симпсоны': 30, 'Синий трактор': 31, 'Смешарики': 32, 'Сумка': 33, 'Трансформеры': 34, 'Фиксики': 35, 'Финник': 36, 'Царевны': 37, 'Цветняшки': 38, 'Чебурашка': 39, 'Черепашки Ниндзя': 40, 'Чик-Чирикино': 41, 'Чуддики': 42, 'Чучело-Мяучело': 43, 'Щенячий патруль': 44}


## Функции обработки признаков

In [174]:
def feature_processing0(df):
    """В модель подаем только название видео"""

    df['reel_name'] = df['reel_name'].str.replace(r'https?://\S+', ' ', regex=True)
    df['reel_name'] = df['reel_name'].fillna(' ')
    
    features = df['reel_name']
    return features

def feature_processing1(df):
    """В модель подаем только название видео и id канала"""

    df['reel_name'] = df['reel_name'].str.replace(r'https?://\S+', ' ', regex=True)

    df['reel_name'] = df['reel_name'].fillna(' ')
    df['yt_channel_id'] = df['yt_channel_id'].fillna(' ')


    features = df['reel_name']+ ' ' +df['yt_channel_id']
    return features

def feature_processing2(df):
    """В модель подаем только описание видео и название канала"""

    df['text'] = df['text'].str.strip('[]')
    df['text'] = df['text'].str.replace(r'https?://\S+', ' ', regex=True)
    df['text'] = df['text'].fillna(' ')
    df['yt_channel_name'] = df['yt_channel_name'].fillna(' ')

    features = df['text'] + ' ' +df['yt_channel_name']
    return features



In [146]:
def feature_processing3(df):
    """В модель подаем только название видео и название канала/id канала"""

    df['reel_name'] = df['reel_name'].str.replace(r'https?://\S+', ' ', regex=True)
    df['reel_name'] = df['reel_name'].fillna(' ')
    df['yt_channel_id'] = df['yt_channel_id'].fillna(' ')
    df['yt_channel_name'] = df['yt_channel_name'].fillna(' ')

    # Функция для формирования features
    def get_features(row):
        language = row['language']
        yt_channel_id = row['yt_channel_id']
        yt_channel_name = row['yt_channel_name']
        reel_name = row['reel_name']

        if language not in ['', 'None', None]:  # Проверяем значение language
            return yt_channel_name+' '  + yt_channel_id 
        else:
            return reel_name + ' ' + yt_channel_id

    features = df.apply(get_features, axis=1)

    return features

def feature_processing4(df, mapping):
    """В модель подаем  название видео + найденный проект (если он содержится в описании видео)"""
  
    df['text'] = df['text'].fillna(' ')
    df['text'] = df['text'].str.replace(r'https?://\S+', ' ', regex=True)
    df['text_short'] = df['text'].str[:250]
    df['reel_name'] = df['reel_name'].str.replace(r'https?://\S+', ' ', regex=True)

    
    # Функция для формирования features
    def get_features(row):
        language = row['language']
        yt_channel_id = row['yt_channel_id']
        yt_channel_name = row['yt_channel_name']
        reel_name = row['reel_name']
        text = row['text']
        
        found_values = []
        for key in mapping.keys():
            # Создаем регулярное выражение для поиска ключа с учетом пробелов и знаков
            pattern = r'\b' + r'\s*.*\s*'.join(re.escape(word) for word in key.split()) + r'\b'
            if re.search(pattern, text, re.IGNORECASE):
                found_values.append(key)

       
        found_string = ' '.join(found_values)

        if language not in ['', 'None', None]:  
            return f"{reel_name} {yt_channel_name} {yt_channel_id} {found_string}".strip()
        else:
            return f"{reel_name} {yt_channel_id} {found_string}".strip()


    # Применяем функцию к DataFrame и создаем новый столбец features
    features = df.apply(get_features, axis=1)

    return features

Есть проблема с иностранными рилс. На подумать
Также есть избыточность в поле text, может без него?

In [175]:
#pd.set_option('display.max_rows', 200)
features = feature_processing1(df)


In [176]:
features_train, features_test, target_train, target_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42, stratify=target_encoded)
features_train.shape

target_train = target_train.squeeze()
target_test = target_test.squeeze()

# Обучение модели и подбор параметров

In [177]:


# Пайплайн для RandomForestClassifier
rf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(random_state=42))
])


rf_param_grid = {
    'rf__n_estimators': [150, 175, 200],
    'rf__min_samples_split': [2, 3, 5],
}

f1_scorer = make_scorer(f1_score, average='macro')  
rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv=5, n_jobs=-1, scoring=f1_scorer)
rf_grid_search.fit(features_train, target_train)

print("Best parameters for Random Forest:", rf_grid_search.best_params_)
print("Best score for Random Forest (F1 macro):", rf_grid_search.best_score_)
rf_model = rf_grid_search.best_estimator_



/opt/anaconda3/envs/practicum/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best parameters for Random Forest: {'rf__min_samples_split': 5, 'rf__n_estimators': 175}
Best score for Random Forest (F1 macro): 0.7568194950137468


In [150]:
rf_predictions = rf_model.predict(features_test)

print("Random Forest Classifier Report:")
print("Accuracy:", accuracy_score(target_test, rf_predictions))

#print("Уникальные классы в target_test:", np.unique(target_test))
#print("Уникальные классы в rf_predictions:", np.unique(rf_predictions))

print(classification_report(target_test, rf_predictions, zero_division=1))

Random Forest Classifier Report:
Accuracy: 0.9937069579672286
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       1.00      0.86      0.92         7
           2       1.00      0.78      0.88        23
           3       1.00      1.00      1.00     15646
           4       1.00      0.83      0.91         6
           5       1.00      0.62      0.77         8
           6       1.00      0.75      0.86         8
           7       1.00      1.00      1.00       106
           8       1.00      0.96      0.98        71
           9       0.75      1.00      0.86         6
          10       1.00      1.00      1.00       118
          11       0.71      0.71      0.71         7
          12       1.00      0.99      0.99        76
          13       0.95      0.91      0.93        46
          14       1.00      0.97      0.99        38
          15       0.80      0.80      0.80         5
          16       

In [151]:
mapping

{'Cry babies magic tears': 0,
 'Enchantimals (Эншантималс)': 1,
 'My little pony': 2,
 'none': 3,
 'Акуленок': 4,
 'Барбоскины': 5,
 'Бременские музыканты': 6,
 'Буба': 7,
 'Бэтмен': 8,
 'Вспыш': 9,
 'Говорящий Том': 10,
 'Губка Боб': 11,
 'Енотки': 12,
 'ЖилаБыла Царевна': 13,
 'Зебра в клеточку': 14,
 'Котик Мормотик': 15,
 'Кошечки собачки': 16,
 'Крутиксы': 17,
 'Кукутики': 18,
 'Лунтик': 19,
 'Малышарики': 20,
 'Маша и медведь': 21,
 'Мини-мишки': 22,
 'Ну_погоди каникулы': 23,
 'Оранжевая корова': 24,
 'Паровозики Чаттингтон': 25,
 'Пороро': 26,
 'Приключения Пети и Волка': 27,
 'Простоквашино': 28,
 'Свинка Пеппа': 29,
 'Симпсоны': 30,
 'Синий трактор': 31,
 'Смешарики': 32,
 'Сумка': 33,
 'Трансформеры': 34,
 'Фиксики': 35,
 'Финник': 36,
 'Царевны': 37,
 'Цветняшки': 38,
 'Чебурашка': 39,
 'Черепашки Ниндзя': 40,
 'Чик-Чирикино': 41,
 'Чуддики': 42,
 'Чучело-Мяучело': 43,
 'Щенячий патруль': 44}

# Обучим модель на всех данных

In [152]:
vectorizer = rf_grid_search.best_estimator_.named_steps['tfidf']
features_tfidf = vectorizer.transform(features) 
rf_model_2 = RandomForestClassifier(n_estimators=175, max_depth=None, min_samples_split=5, random_state=42)
rf_model_2.fit(features_tfidf, target_encoded)

/opt/anaconda3/envs/practicum/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(min_samples_split=5, n_estimators=175, random_state=42)

In [153]:
model = rf_model_2



# Сабмит

In [154]:
#сделаем submit

#Submission must have 55608 rows

t = []

with open('data/test.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)

    for row in reader:
        # Проверяем, что первая колонка существует и начинается с "20"
        if row and row[0].startswith("20"):  # Проверка на пустую строку
            t.append(row)

# Создаем DataFrame из списка корректных строк
submit = pd.DataFrame(t, columns=header)

# Вывод информации о DataFrame
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55608 entries, 0 to 55607
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             55608 non-null  object
 1   reel_name        55608 non-null  object
 2   yt_reel_id       55608 non-null  object
 3   url              55608 non-null  object
 4   text             55608 non-null  object
 5   seconds          55572 non-null  object
 6   is_shorts        55572 non-null  object
 7   broadcast        55572 non-null  object
 8   yt_channel_id    55572 non-null  object
 9   yt_channel_name  55572 non-null  object
 10  yt_ch_url        55572 non-null  object
 11  yt_channel_type  55572 non-null  object
 12  flag_closed      55572 non-null  object
 13  international    55572 non-null  object
 14  language         55572 non-null  object
dtypes: object(15)
memory usage: 6.4+ MB


In [155]:
features_submit = feature_processing1(submit)
features_submit[4]

'СЛИШКОМ МАЛЕНЬКИЙ ПОРТАЛ В ЭНД В МАЙНКРАФТ ТОЛЬКО 1 ИГРОКОВ MINECRAFT ТРОЛЛИНГ ЛОВУШКА UCSmqZkD74x3TLt6eRM5GhTg'

In [156]:

features_submit_tfidf = vectorizer.transform(features_submit)  

In [157]:
rf_predictions_submit = model.predict(features_submit_tfidf)


In [158]:
rf_predictions_submit_decoded = label_encoder.inverse_transform(rf_predictions_submit)

rf_predictions_submit_decoded = pd.DataFrame(rf_predictions_submit_decoded, columns=['cartoon'])
len(rf_predictions_submit_decoded['cartoon'].unique())

43

In [159]:
submission = pd.concat([submit['yt_reel_id'], rf_predictions_submit_decoded], axis=1)

In [160]:
submission

,yt_reel_id,cartoon
0,RBw4j0aMWyM,none
1,5wTfpmlZe5k,none
2,Fd9YA-oDjSo,none
3,NXCHY_J-5v0,none
4,XM-GRekoYzs,none
...,...,...
55603,z8QQtWqr7lY,none
55604,VMBC_ahkjjU,none
55605,zwSu3nUWQKM,none
55606,yieUVOAHZiA,none


In [161]:
submission.to_csv('submission13.csv', index=False)



In [162]:
#проверка с лучшим саюмитом
# Загрузка данных из CSV файлов
df1 = pd.read_csv('submission1.csv')
df2 = submission

# Сравнение данных
# Найдем строки, которые есть в df1, но отсутствуют в df2
diff1 = df1[~df1.apply(tuple, 1).isin(df2.apply(tuple, 1))]

# Найдем строки, которые есть в df2, но отсутствуют в df1
diff2 = df2[~df2.apply(tuple, 1).isin(df1.apply(tuple, 1))]

# Объединяем различия
differences = pd.concat([diff1, diff2])

# Выводим различия
print("Различия между файлами:")
print(diff1)


Различия между файлами:
        yt_reel_id        cartoon
500    pDU4xJPnspI           none
711    Xxj94MEqPW0           none
2274   jMBKxOv-rQU           none
2439   kOjj3BAW5Ss           none
2460   3B9UCYP5Dq0  Простоквашино
...            ...            ...
48738  TgeRLCujcfg           none
48912  dDkd4TxfSeM  Простоквашино
50531  ZmWvepR7wlo           Буба
52681  jA3-cTdfsfk           none
53743  fkhjUxzwITo           none

[89 rows x 2 columns]


In [163]:
print(diff2)

        yt_reel_id                   cartoon
500    pDU4xJPnspI                 Губка Боб
711    Xxj94MEqPW0                    Бэтмен
2274   jMBKxOv-rQU            Маша и медведь
2439   kOjj3BAW5Ss                   Фиксики
2460   3B9UCYP5Dq0                      none
...            ...                       ...
48738  TgeRLCujcfg            Чучело-Мяучело
48912  dDkd4TxfSeM  Приключения Пети и Волка
50531  ZmWvepR7wlo                      none
52681  jA3-cTdfsfk          ЖилаБыла Царевна
53743  fkhjUxzwITo                  Акуленок

[89 rows x 2 columns]


In [164]:


joblib.dump(model, 'best_model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

print("Модель и векторизатор успешно сохранены.")

Модель и векторизатор успешно сохранены.
